# PV181 Seminar 01 - RNG (python)

This notebook contains python code for several tasks treated in this seminar. 

# PRNG 

# Task 1: determinsm of PRNG

We will work with PRNG implemented in [random](https://docs.python.org/3/library/random.html) package. See first 4 methods (`seed, setstate, getstate, randbytes`) in the documentation. 
 1. Import **random** package.
 2. Generate (and print) 10 random bytes.  
 3. Print out bytes in hexadecimal form (use `.hex()` method of bytes). Execute cell 2x - use Run or Ctrl+Enter. 
 4. Seed the generator so that it will produce 10 bytes '73a9bef499bbf4dca4f2' (try different seeds 0, 1, 2). Execute cell 2x to check that result is the same. 
 4. Get state of the PRNG right after the seeding (before generation of bytes).  
 5. Set the state of PRNG and generate the same result as in step 4. 

# LCG
Standard PRNG functions are very fast but also very weak. 
 * In python, PRNG [implemented](https://svn.python.org/projects/python/branches/release32-maint/Lib/random.py) in random module is [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) with state formed by 625 32-bit integers. 
 * In other languages (C, Java, Rust) LCG is typically used. Internal state of LCG is **single** value (state) updated iterativelly as $$state = (state*a+c) \pmod m.$$ Overview of constants `a,c,m` used by the LCG for several languages can be found [here](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 <span style="color:red">In LCG, state (new or old) is typically returned as generated random value!!</span>

# Task 2: ANSI C 
Following code was taken from [ANSI C standard](http://www.open-std.org/jtc1/sc22/wg14/www/docs/n1256.pdf#page=324) and simplified to other portable implementation (according to implementation of [rand()](https://code.woboq.org/userspace/glibc/stdlib/random_r.c.html#__random_r)) of seeding function `srand` and function for generation `rand()`. 

```
static unsigned long int next = 1;

void srand(unsigned int seed)
{
    next = seed;
}

int rand(void) // RAND_MAX assumed to be 32767
{
    return next = (next * 1103515245 + 12345) & 0x7fffffff;
}
```
 1. Implement pythonic version ANSI C standard PRNG - implement it as `class PRNG` below. Use constants $a=1103515245, c=12345, m=2^{31}$ from [LCG wikipedia](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 2. Generate 10 values with LCG seeded by 1.  
 3. Generator ANSI C generated sequence $12345, 1406932606$. Find next two values  $6545..., 1449...$.  
 Seed the generator and simply continue in generation. 
 4. Find previous two values. Seed the generator with the "oldest" and check that new generated value is equal to 0.    
  - New state is computed from previous by: $$new = old*a+c \pmod m$$ 
  - How $old$ value can by computed from new one? 
  - Compute previous one using  division  $/a \pmod m$ is equivalent to  `*pow(a,-1,m)`.
 
 [comment $$old = (new-c)/a \pmod m$$]::


In [1]:
class PRNG:
    def __init__(self):
        pass
    def srand(self, seed):
        pass

    def rand(self):
        pass
        
ansi_rand = PRNG()

# Task 3: hashing, test vectors  
Verify that hash functions functions are implemented correctly. There are examples of input-output pairs (called test vectors) computed by reference (original) implementation. Two test vectors for SHA1, SHA256 hash functions and empty message are following:
 * SHA1("")=da39a3ee5e6b4b0d3255bfef95601890afd80709.
 * SHA256("")=e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855.
 1. Check test correctness of implementation.  
 <span style="color:red">Implementations of both hash functions expect bytes as input not string!</span> See `Bytes Objects` in python [Built-in Types](https://docs.python.org/3/library/stdtypes.html).
 2. Google both hashes. Find (google, check the standard) other test vectors .  

In [2]:
from cryptography.hazmat.primitives import hashes

def SHA1(message: bytes):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(message)
    return digest.finalize() 

def SHA256(message: bytes):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(message)
    return digest.finalize() 

# Task 4: CSPRNG
Hash functions `SHA1, SHA256` can be used as one-way functions (hard to invert) to create CSPRNG. One-wayness of SHA1 doesn't allow to compute previous state of RNG. On the other hand current implementation (below) returns some of the next states of CSPRNG as generated random value (`SHA1(self.state)` is returned and for update of `self.state`).     
 1. `csprng` generated three blocks (`fbfb5caad5c041b8470733a232858348e385e108`, `88c3f9b87bdd514831223607f0bdfb2d2d76a68f`, `cc1909ab7e31f8add561cd39929744f5ff3fcdbb`) in a row. Seed the generator and find next block `4b98cd1ccc618d1dbdbbea7c6f2a2d8053f47f8a`. 
 2. Change the implementation so that attack is not longer possible (use other hash function). 

In [1]:
class CSPRNG:
    def __init__(self, state_size = 2):
        pass

    def srand(self, seed: bytes):
        pass
    def rand(self):
        self.state = SHA1(self.state)  
        return SHA1(self.state)
    
csprng = CSPRNG()

# TRNG

# Sources: dev/random, dev/urandom
These two files provide secure way to generate random bytes! 
Reading from dev/urandom can by done using following functions: 

In [19]:
import os
import secrets 
os.urandom(10)
secrets.token_bytes(10)
os.getrandom(10)

b'\xc2\xdf\x92\xd6oG\xe8\xb3{x'

Files **dev/random**, **dev/urandom** can be also opened as binary file for reading.  
Then you can read specified number of bytes e.g. 10. 

In [52]:
random_source = open("/dev/random", "rb")
random_source.read(10)

b'\xc1\xab\xce\x9e\xe7\xb1n\xa5\x15\x06'

# Task 5: Testing correlation of bits
 1. Implement function `histogram(rnd_bytes, i, j)` that computes histogram of combination of bits (`i`-th and `j`-th bits of each byte). The function should return 4 frequencies for all bytes in `rnd_bytes` array. Frequencies will correspond to counts of how many bytes have `i`-th and `j`-th bit equal to combination 00,01,10 or 11.


In [4]:
import os
def histogram(rnd_bytes, i, j):
    hist = {}
    return hist 



 2. Verify that for arbitrary params `i,j` and size of generated block the frequencies are roughly equal.  

In [5]:
rnd_bytes = os.urandom(1000)
histogram(rnd_bytes, 0, 1)

{}

 3. Generate random bytes using ansi_rand = generate integers, apply modulo, transform to bytes (use `bytes()`).
 4. Find params `i,j` where all the frequencies are exacly the same.  
 <span style="color:red"> Generator with such perfect results is also problematic! </span>   
 Can we predict some (next) bits with better probability than 50% ?

In [6]:
ansi_rand.srand(2)
rnd_bytes = bytes([])
histogram(rnd_bytes, 0, 1)

{}